# Imports

In [1]:
from numpy.random import seed
seed(888)
from tensorflow import random
random.set_seed(404)

In [2]:
import os
import numpy as np
import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from time import strftime

Instructions for updating:
non-resource variables are not supported in the long term


# Constants

In [3]:
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'

LOGGING_PATH = 'tensorboard_mnist_digit_logs/'

NR_CLASSES = 10
VALIDATION_SIZE= 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = (IMAGE_WIDTH*IMAGE_HEIGHT*CHANNELS)

# Get the Data

In [4]:
%%time

y_train_all = np.loadtxt(Y_TRAIN_PATH,delimiter=',',dtype=int) #read the text file and convert to numpy array

CPU times: total: 0 ns
Wall time: 9.53 ms


In [5]:
y_train_all.shape

(60000,)

In [6]:
y_test = np.loadtxt(Y_TEST_PATH,delimiter = ',',dtype=int)

In [7]:
y_test.shape

(10000,)

In [8]:
%%time

x_train_all = np.loadtxt(X_TRAIN_PATH,delimiter=',',dtype=int)

CPU times: total: 7.42 s
Wall time: 12 s


In [9]:
%%time

x_test = np.loadtxt(X_TEST_PATH,delimiter=',',dtype=int)

CPU times: total: 1.61 s
Wall time: 2.61 s


In [10]:
x_test.shape

(10000, 784)

# Explore

In [11]:
x_train_all.shape

(60000, 784)

In [12]:
x_train_all[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [13]:
y_train_all[:5] # five first entry, our targer is sparse because has int for classes

array([5, 0, 4, 1, 9])

# Data Preprocessing

In [14]:
# Rescale

x_train_all, x_test = x_train_all/255.0, x_test/255.0

#### Convert target values to one-hot encoding

In [15]:
values = y_train_all[:5]
np.eye(10)[values]  # array element indexing  #the values in the square brackets acts as the index array.
# we're using this entire array as an index and we're pulling out several of the rows from the identity matrix.

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [16]:
np.eye(10) # why 10? because we have got 10 different labels in our dataset 0-9

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [17]:
np.eye(10)[9]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [18]:
values

array([5, 0, 4, 1, 9])

In [19]:
values[2]

4

In [20]:
y_train_all = np.eye(NR_CLASSES)[y_train_all]

In [21]:
y_train_all.shape

(60000, 10)

In [22]:
y_test = np.eye(NR_CLASSES)[y_test]
y_test.shape

(10000, 10)

### Create validation dataset from training data

**Challenge:** Split the training dataset into a smaller training dataset and a validation dataset for the features and the labels. Create four arrays: ```x_val```, ```y_val```, ```x_train```, and ```y_train``` from ```x_train_all``` and ```y_train_all```. Use the validation size of 10,000. 

In [23]:
x_val =  x_train_all[:VALIDATION_SIZE] # first 10,000 value
y_val =  y_train_all[:VALIDATION_SIZE]

In [24]:
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

In [25]:
y_train.shape

(50000, 10)

# Setup Tensorflow Graph

- To create a tensor we need, data types and a shape

In [26]:
# set features and labels as placeholders with variable batch size and data type tf.float32
#X = tf.keras.Input(shape=(None, TOTAL_INPUTS), dtype=tf.float32) # features
#Y = tf.keras.Input(shape=(None, NR_CLASSES), dtype=tf.float32)  # labels


# set features and labels as placeholders with variable batch size and data type tf.float32
X = tf.placeholder(tf.float32, shape=[None, TOTAL_INPUTS], name='X')
Y = tf.placeholder(tf.float32, shape=[None, NR_CLASSES], name='labels')

### Neural Network Architecture

#### Hyperparameters

In [27]:
nr_epochs = 50
learning_rate = 1e-3    # le-4 = 0.0001

# Nr neurons and layers
# the output layers is getting 10

n_hidden1 = 512
n_hidden2 = 64

## First Hidden Layer

In [28]:
# initialized weight for the first hidden layers
initial_w1 = tf.random.truncated_normal(shape=[TOTAL_INPUTS,n_hidden1], stddev=0.1, seed=42) # nr inputs and nr layers
w1 = tf.Variable(initial_value=initial_w1) #  variables are used to hold and update the parameters  

In [29]:
# initialized bias for the first hidden layers
initial_b1 = tf.constant(value=0, shape=[n_hidden1])  #value= zero to initializing biases.
b1 = tf.Variable(initial_value=initial_b1)

In [30]:
# calculates the input to the first hidden layer
# X -> raw input
layer1_in = tf.matmul(X, w1) + tf.cast(b1, tf.float32) #matrix multiplication between the input tensor X and the weight tensor w1
#layer1_in = tf.keras.layers.Lambda(lambda x: tf.add(tf.matmul(X, w1), tf.cast(b1, tf.float32)), name='layer1_in')

In [31]:
# output of this layers
layer1_out = tf.nn.relu(layer1_in)

## Second Hidden Layer

In [32]:
# initialized weight for the second hidden layers
# in second layer instead of total input, get the second hidden layer
initial_w2 = tf.random.truncated_normal(shape=[n_hidden1,n_hidden2], stddev=0.1, seed=42)
w2 = tf.Variable(initial_value=initial_w2)

# initialized bias for the second hidden layers
initial_b2 = tf.constant(value=0, shape=[n_hidden2])
b2 = tf.Variable(initial_value=initial_b2)

# what the features were that were going into the second hidden layer. output of second layers 
layer2_in = tf.matmul(layer1_out, w2) +  tf.cast(b2, tf.float32)

# output of this layers
layer2_out = tf.nn.relu(layer2_in)

## Output Layer

In [33]:
# weight
initial_w3 = tf.random.truncated_normal(shape=[n_hidden2,NR_CLASSES], stddev=0.1, seed=42)
w3 = tf.Variable(initial_value=initial_w3)

# bias
initial_b3 = tf.constant(value=0, shape=[NR_CLASSES])
b3 = tf.Variable(initial_value=initial_b3)

# layers input and output 
layer3_in = tf.matmul(layer2_out, w3) +  tf.cast(b3, tf.float32)
output = tf.nn.softmax(layer3_in)

#  Tensorboard Setup

In [34]:
folder_name = strftime("Model1_%Y-%m-%d_%H-%M-%S")  #folder_name = f'Model 1 at {strftime("%H-%M")}'

directory = os.path.join(LOGGING_PATH, folder_name)

try:
    os.makedirs(directory)
    print(f"Successfully created directory: {directory}")
except OSError as exception:
    print(f"Failed to create directory: {directory}")
    print(exception.strerror)


Successfully created directory: tensorboard_mnist_digit_logs/Model1_2023-05-06_17-03-02


# Loss, Optimisation & Metrics

### Defining Loss function

- Loss = Calculate the error between the actual output and predicted output


- Logit = the output from the output layer


- label = are going to be the actual labels (Y)

In [35]:
# we are training data in the batches that's why need to take the average of the losses

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output))

### Defining Optimizers

In [36]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

- our output is a operation which update the variable, i.e., weights and biases

### Accuracy Metrics

- Accuracy =  compare "prediction" and "true labels" and we need to check if they're equal.


- Once we know if they are equal, then we have a correct prediction.


- And once we know how many correct predictions we have, we can work out the accuracy of our model.

In [37]:
# compare two quantities in this case the output and Y 

correct_pred = tf.equal(tf.argmax(output, axis=1), tf.argmax(Y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32)) # the average across all the batches

In [38]:
tf.summary.scalar('accuracy', accuracy)

<tf.Tensor 'accuracy:0' shape=() dtype=string>

# Train our model 

- involves of starting a session, batching our data and finally running the trainig loop

## Run Sessions

In [39]:
# session object encapsulate the environment where all operation and calculation take placeand executed
sess = tf.Session()

### Setup Filewriter and Merge Summeries

In [40]:
#  FileWriter is an object that can be used to write the summary data that can be visualized with TensorBoard.
merged_summary = tf.summary.merge_all()

train_writer = tf.summary.FileWriter(directory + '/train')
train_writer.add_graph(sess.graph)

#validation_writer = tf.summary.FileWriter(directory + '/validation')

In [41]:
# Initialized all the variables 

init = tf.global_variables_initializer()
sess.run(init) # run session

# these three line of code get us all set up

In [42]:
w1.eval(sess)

array([[-0.02807751, -0.01377521, -0.06763297, ..., -0.02663724,
         0.02861341, -0.05550233],
       [-0.16205125, -0.18859725, -0.03102448, ..., -0.0820701 ,
        -0.03345905, -0.02453214],
       [ 0.12582639, -0.16444902,  0.13603579, ..., -0.09897225,
        -0.09923435,  0.1451435 ],
       ...,
       [-0.04471838, -0.09593774, -0.08978765, ...,  0.04240045,
        -0.18997248,  0.00134785],
       [ 0.03215451,  0.04336654, -0.18240118, ...,  0.08296242,
        -0.10039439, -0.12682591],
       [ 0.08766606, -0.15083945,  0.08048793, ...,  0.07548849,
        -0.04359084, -0.11031353]], dtype=float32)

In [43]:
b3.eval(sess) # output biases

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [44]:
# fisrt we need to spilit our training data into smaller components
# because we want to be able to train our model on batches of 1000 samples at a time.

## Batching the Data

In [45]:
size_of_batch = 1000

In [46]:
num_examples = y_train.shape[0]
nr_iterations = int(num_examples/size_of_batch)

index_in_epoch= 0 # help where one batch end and another one start  
# 1st 0-> 999 2nd -> 1000 ->1999,...until 50000 examples in our training

In [47]:
# defining function for going from one batch to another batch
#batch size -> how big the batch be
# data and labels --> x, y respectively

In [48]:
def next_batch(batch_size, data, labels):
    
    global num_examples   # hold the variables that is outside of this function
    global index_in_epoch  
    
    start = index_in_epoch   # inx 0 then will be update
    index_in_epoch += batch_size   # index_in_epoch = index_in_epoch + batch_size
    
    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch = batch_size
    
    end = index_in_epoch
    
    return data[start:end], labels[start:end]  # first time you get value between 0 and 999

### Training Loop

In [49]:
for epoch in range(nr_epochs):

    # ============= Training Dataset =========
    for i in range(nr_iterations):

        batch_x, batch_y = next_batch(batch_size=size_of_batch, data=x_train, labels=y_train)

        feed_dictionary = {X:batch_x, Y:batch_y}  


        sess.run(optimizer, feed_dict=feed_dictionary)


    acc, loss_ = sess.run([accuracy,loss], feed_dict=feed_dictionary)
    
    
    acc_sum = tf.Summary(value=[tf.Summary.Value(tag="accuracy", simple_value=acc),])
    train_writer.add_summary(acc_sum, epoch)
    
    loss_sum = tf.Summary(value=[tf.Summary.Value(tag="loss", simple_value=loss_),])   
    train_writer.add_summary(loss_sum, epoch)

    print(f'Epoch {epoch} \t| Training Accuracy = {acc}')
    
    
print('Done training!')

Epoch 0 	| Training Accuracy = 0.8600000143051147
Epoch 1 	| Training Accuracy = 0.8659999966621399
Epoch 2 	| Training Accuracy = 0.8709999918937683
Epoch 3 	| Training Accuracy = 0.8740000128746033
Epoch 4 	| Training Accuracy = 0.8769999742507935
Epoch 5 	| Training Accuracy = 0.9700000286102295
Epoch 6 	| Training Accuracy = 0.984000027179718
Epoch 7 	| Training Accuracy = 0.9850000143051147
Epoch 8 	| Training Accuracy = 0.9860000014305115
Epoch 9 	| Training Accuracy = 0.9869999885559082
Epoch 10 	| Training Accuracy = 0.9869999885559082
Epoch 11 	| Training Accuracy = 0.9879999756813049
Epoch 12 	| Training Accuracy = 0.9890000224113464
Epoch 13 	| Training Accuracy = 0.9900000095367432
Epoch 14 	| Training Accuracy = 0.9890000224113464
Epoch 15 	| Training Accuracy = 0.9900000095367432
Epoch 16 	| Training Accuracy = 0.9900000095367432
Epoch 17 	| Training Accuracy = 0.9900000095367432
Epoch 18 	| Training Accuracy = 0.9900000095367432
Epoch 19 	| Training Accuracy = 0.99099999

# Reset for the Next Run

In [50]:
train_writer.close()
sess.close()
tf.reset_default_graph()